In [10]:
import os 
import pandas as pd
import time
import requests 
import neo4j
from neo4j import GraphDatabase

from dotenv import load_dotenv

load_dotenv()

True

## Scraping

Discourse API docs here: https://docs.discourse.org/

### Posts

First scrape posts.

Desired JSON:

```
{
  "id": "post_id",
  "text": "cooked_content_from_api",
  "author": {
    "username": "author_username",
    "id": "author_id"
  },
  "tags": ["tag1", "tag2"],
  "url": "post_url",
  "topic_id": "associated_topic_id",
  "responses": [
    {
      "id": "response_post_id",
      "text": "cooked_content_from_api",
      "author": {
        "username": "author_username",
        "id": "author_id"
      }
    }
  ]
}
```


In [2]:
import requests
import time

def fetch_categories(base_url):
    """Fetch all categories from the forum."""
    url = f"{base_url}/categories.json"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            return data.get('category_list', {}).get('categories', [])
        except ValueError:
            print("Failed to decode JSON.")
            return []
    else:
        print(f"Failed to fetch categories: {response.status_code}")
        return []

def fetch_topics(base_url, category_slug):
    topics = []
    page = 0
    while True:
        url = f"{base_url}/c/{category_slug}.json?page={page}"
        response = requests.get(url)
        if response.status_code != 200:
            break
        data = response.json()
        fetched_topics = data.get("topic_list", {}).get("topics", [])
        if not fetched_topics:
            break
        topics.extend(fetched_topics)
        page += 1
    return topics

def fetch_topic_details(base_url, topic_id):
    url = f"{base_url}/t/{topic_id}.json"
    response = requests.get(url)
    time.sleep(.5)  # Respectful delay to avoid hitting rate limits
    if response.status_code != 200:
        return None
    return response.json()

def organize_post_data(base_url, topic_details):
    posts_data = []
    posts = topic_details.get("post_stream", {}).get("posts", [])
    for post in posts:
        post_data = {
            "text": post.get("cooked", ""),
            "title": topic_details.get("title", ""),
            "author": post.get("username", ""),
            "tags": topic_details.get("tags", []),
            "url": f"{base_url}/t/{topic_details.get('id')}/{post.get('post_number')}"
        }
        responses = [response for response in posts if response.get("reply_to_post_number") == post.get("post_number")]
        post_data["responses"] = [{
            "text": response.get("cooked", ""),
            "author": response.get("username", "")
        } for response in responses]
        posts_data.append(post_data)
    return posts_data



In [ ]:
arb_base_url = "https://forum.arbitrum.foundation"

In [1]:
categories = fetch_categories(arb_base_url)
categories[0]

NameError: name 'fetch_categories' is not defined

In [41]:
def organize_post_data(base_url, topic_details, category_name):
    """Organize post data into the desired JSON structure, including category and topic info."""
    posts_data = []
    posts = topic_details.get("post_stream", {}).get("posts", [])
    for post in posts:
        post_data = {
            "text": post.get("cooked", ""),
            "title": topic_details.get("title", ""),
            "author": post.get("username", ""),
            "tags": topic_details.get("tags", []),
            "category": category_name,  # Include category name
            "topic_id": topic_details.get("id"),  # Include topic ID
            "url": f"{base_url}/t/{topic_details.get('id')}/{post.get('post_number')}"
        }
        # Find responses to this post
        responses = [response for response in posts if response.get("reply_to_post_number") == post.get("post_number")]
        post_data["responses"] = [{
            "text": response.get("cooked", ""),
            "author": response.get("username", "")
        } for response in responses]
        posts_data.append(post_data)
    return posts_data


In [36]:
all_posts = organize_post_data(arb_base_url, c

5966